## Implementation of the main steps for chemical data curation following the paper "Trust, But Verify: On the Importance of Chemical Structure Curation in Cheminformatics and QSAR Modeling Research" from 2010 (D.Fourches, ...)

Link: https://pubmed.ncbi.nlm.nih.gov/20572635/


In [ ]:
#import modules
from rdkit import Chem
from opencadd.compounds.standardization import convert_format,remove_fragments,disconnect_metals,detect_inorganic,remove_salts

In [11]:
# INITIAL LIST OF SMILES
# import test smiles dataset
test_smiles = "NC(CC(=O)O)C(=O)[O-].O.O.[Na+]"
test_smiles2 = "C(C(=O)[O-])(Cc1n[n-]nn1)(C[NH3+])(C[N+](=O)[O-])"
test_smiles3= "CN(C)C.Cl.Cl.Br"

### Removal of Inorganics and Mixtures

Since molecular descriptors can only be computed for organic compunds, all inorganic compunds must be removed before the descriptors are calculated. (Chapter 2.1. Fourches 2010)

For the flagging and following removal of compounds containing inorganic molecules, we can use the function `detect_inorganic`. This function returns a boolean value of "True" when it finds a inorganic molecule. We can run this flagging in a pre-processing step of the data, and discard those compounds. 
Inorganic compounds are known to have biological effects, like for example toxic effects. Due to their bioactivity we can not distinguish if the recored activity of a mixed compound is caused by it's organic or inorganic part. Therefore the entry is useless and can be discarded. ! THIS SHOULD BE LOGGED AND MANUAL CURATION SHOULD BE ENABLED !
An alternate and easy way would be that every SMILES  is undertaken a substring search, where a match of a inorganic compound pattern (search pattern set should be defined) would be  flagged.

Due to the fact, that the treatment is not as simple as it apprears the paper (Fourches, 2010) recommends to delete records containing mixtures. ! THIS AGAIN CAN BE LOGGED AND MANUAL CURATION CAN BE DONE WITH THIS SET ! The ease up the curation various filtering functions can be implemented to help decide which to keep and which to discard. Three types of mixtures are described. ! CHECK IF IMPLEMENTATION WOULD BE POSSIBLE EASY AND FAST ! Common and widely used practice is to retain molecules with the highest molecular weight or the largest number of atoms, but the paper (Fourches, 2010) states this might not be the best solution, and further investigation in mixtures should only be done if there is a reason to belive that the biological activity is really caused by the largest molecule and not the mixture itself.

In [12]:
# Removal of mixtures, inorganics (and eventually organometallics)
# Functions detect_inorganic,remove_fragments, disconnect_metals, detect_inorganic again

In [ ]:
# Structural coversion
# Cleaning/removal of salts
# Functions remove_salts

In [ ]:
# Normalization of specific chemotypes
# normalize_molecules

In [ ]:
# Treatment of tautomeric forms
# handle_tautomers

In [ ]:
# Analysis/removal of duplicates

In [ ]:
# Manual inspection